# Learning Progress Prediction (XGBoost)
Notebook này **chỉ dùng XGBoost** để:
- Tạo dữ liệu feature-engineering (đồng bộ với `pipeline.py`).
- Train lại model trên toàn bộ dữ liệu.
- Sinh file dự đoán `submissionn.csv` từ `data/test.csv`.

Gợi ý workflow: chạy notebook này xong rồi mới chạy Streamlit app (`streamlit_app.py`).

## 1) Import & cấu hình
- Chạy notebook tại thư mục gốc project (nơi có `data/`, `pipeline.py`).
- Nếu chạy trên máy không có GPU/CUDA, cứ dùng CPU bình thường.

In [ ]:
from __future__ import annotations

import json
import os
from pathlib import Path
from typing import Iterable, List, Optional, Tuple

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

try:
    import optuna  # optional nhưng khuyến nghị để tune
except Exception:
    optuna = None

# ============================
# Pipeline inline (no imports)
# ============================
FEATURES: List[str] = [
    "TC_DANGKY",
    "SO_NAM_HOC",
    "COVID_ONLINE_THPT",
    "COVID_ONLINE_DH",
    "HIST_AVG_GPA",
    "HIST_AVG_TC_DANGKY",
    "HIST_PASS_RATE",
    "HIST_GPA_STD",
    "TOTAL_FAIL_CREDITS",
    "LAG1_GPA",
    "LAG2_GPA",
    "LAG1_PASS_RATE",
    "TREND_GPA",
    "LOAD_RATIO",
    "SCORE_GAP",
    "DIEM_TRUNGTUYEN",
    "DIEM_CHUAN",
    "NAM_TUYENSINH",
    "PTXT_1",
    "PTXT_100",
    "PTXT_200",
    "PTXT_3",
    "PTXT_402",
    "PTXT_409",
    "PTXT_5",
    "PTXT_500",
    "AVG_SCORE",
]

def _get_current_year(hoc_ky: str) -> int:
    arr = str(hoc_ky).split("-")
    return int(arr[-1])

def _key_hoc_ky(hoc_ky: str) -> Tuple[int, int]:
    # Format: 'HK1 2020-2021'
    hk, year = str(hoc_ky).split()
    hk = int(hk.replace("HK", ""))
    year = int(year.split("-")[0])
    return (year, hk)

def _latest_semester_label(values: Iterable[str]) -> Optional[str]:
    labels = [v for v in pd.Series(list(values)).dropna().astype(str).unique().tolist()]
    if not labels:
        return None
    labels_sorted = sorted(labels, key=_key_hoc_ky)
    return labels_sorted[-1]

def build_main_df(academic_df: pd.DataFrame, admission_df: pd.DataFrame) -> pd.DataFrame:
    """Rebuild the engineered dataset.

    Output contains one row per (MA_SO_SV, HOC_KY) with engineered history features.
    """
    df = pd.merge(academic_df, admission_df, how="left", on="MA_SO_SV")

    # One-hot PTXT (keep stable set as much as possible)
    if "PTXT" in df.columns:
        dummies = pd.get_dummies(df["PTXT"], prefix="PTXT", dtype=float)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(columns=["PTXT"])

    df["SO_NAM_HOC"] = df["HOC_KY"].apply(_get_current_year) - df["NAM_TUYENSINH"]

    df["_key_hocky"] = df["HOC_KY"].map(_key_hoc_ky)
    df = df.sort_values(by=["MA_SO_SV", "_key_hocky"]).drop(columns=["_key_hocky"])

    # History aggregates
    df["HIST_AVG_GPA"] = (
        df.groupby("MA_SO_SV")["GPA"].transform(lambda x: x.shift(1).expanding().mean()).fillna(0)
    )
    df["HIST_AVG_TC_DANGKY"] = (
        df.groupby("MA_SO_SV")["TC_DANGKY"]
        .transform(lambda x: x.shift(1).expanding().mean())
        .fillna(0)
    )

    df["HIST_PASS_RATE"] = (
        df.groupby("MA_SO_SV")["TC_HOANTHANH"].cumsum().shift(1)
        / df.groupby("MA_SO_SV")["TC_DANGKY"].cumsum().shift(1)
    ).fillna(1)

    df["LAG1_GPA"] = df.groupby("MA_SO_SV")["GPA"].shift(1).fillna(0)
    df["LAG2_GPA"] = df.groupby("MA_SO_SV")["GPA"].shift(2).fillna(0)
    df["LAG1_PASS_RATE"] = df.groupby("MA_SO_SV")["HIST_PASS_RATE"].shift(1).fillna(1)
    df["TREND_GPA"] = (df["LAG1_GPA"] - df["HIST_AVG_GPA"]).fillna(0)

    df["PASS_RATE"] = (df["TC_HOANTHANH"] / df["TC_DANGKY"]).replace([np.inf, -np.inf], np.nan)
    df["PASS_RATE"] = df["PASS_RATE"].fillna(0).astype(float)

    df["SCORE_GAP"] = df["DIEM_TRUNGTUYEN"] - df["DIEM_CHUAN"]

    df["HIST_GPA_STD"] = (
        df.groupby("MA_SO_SV")["GPA"].transform(lambda x: x.shift(1).expanding().std()).fillna(0)
    )

    df["FAIL_CREDITS_TEMP"] = df["TC_DANGKY"] - df["TC_HOANTHANH"]
    df["TOTAL_FAIL_CREDITS"] = (
        df.groupby("MA_SO_SV")["FAIL_CREDITS_TEMP"].transform(lambda x: x.shift(1).cumsum()).fillna(0)
    )
    df = df.drop(columns=["FAIL_CREDITS_TEMP"])

    df["LOAD_RATIO"] = df["TC_DANGKY"] / df["HIST_AVG_TC_DANGKY"].replace(0, 1)

    avg_score_map = {
        2025: 6.50,
        2024: 6.57,
        2023: 6.03,
        2022: 6.34,
        2021: 4.97,
        2020: 5.19,
        2019: 4.30,
        2018: 3.79,
        2017: 4.59,
        2016: 4.50,
        2015: 5.25,
    }
    df["AVG_SCORE"] = df["NAM_TUYENSINH"].map(avg_score_map)

    covid_years_thpt = [2020, 2021]
    covid_semesters_dh = [
        "HK1 2019-2020",
        "HK2 2019-2020",
        "HK1 2020-2021",
        "HK2 2020-2021",
        "HK1 2021-2022",
    ]
    df["COVID_ONLINE_THPT"] = df["NAM_TUYENSINH"].isin(covid_years_thpt).astype(int)
    df["COVID_ONLINE_DH"] = df["HOC_KY"].isin(covid_semesters_dh).astype(int)

    # Ensure any missing PTXT dummy columns exist for downstream.
    for col in [c for c in FEATURES if c.startswith("PTXT_")]:
        if col not in df.columns:
            df[col] = 0.0

    # Fill other feature columns if missing
    for col in FEATURES:
        if col not in df.columns:
            df[col] = 0.0

    return df

def build_prediction_frame(
    uploaded_df: pd.DataFrame,
    main_df: pd.DataFrame,
    preferred_last_hk: str = "HK2 2023-2024",
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Build (ready_df, pred_df) aligned to FEATURES."""
    if "MA_SO_SV" not in uploaded_df.columns or "TC_DANGKY" not in uploaded_df.columns:
        raise ValueError("CSV upload phải có cột MA_SO_SV và TC_DANGKY.")

    last_hk = preferred_last_hk
    if last_hk not in set(main_df["HOC_KY"].astype(str).unique().tolist()):
        inferred = _latest_semester_label(main_df["HOC_KY"].astype(str).values)
        if inferred is None:
            raise ValueError("Không xác định được học kỳ gần nhất từ dữ liệu lịch sử.")
        last_hk = inferred

    last_snapshot = main_df[main_df["HOC_KY"].astype(str) == str(last_hk)].copy()

    # Merge: uploaded test-like data + last snapshot
    pred_df = pd.merge(uploaded_df, last_snapshot, how="left", on="MA_SO_SV", suffixes=("_x", "_y"))

    # Update SO_NAM_HOC for next term
    if "SO_NAM_HOC" in pred_df.columns:
        pred_df["SO_NAM_HOC"] = pd.to_numeric(pred_df["SO_NAM_HOC"], errors="coerce").fillna(0) + 1
    else:
        pred_df["SO_NAM_HOC"] = 0

    pred_features = [
        "TC_DANGKY_x",
        "SO_NAM_HOC",
        "COVID_ONLINE_THPT",
        "COVID_ONLINE_DH",
        "HIST_AVG_GPA",
        "HIST_AVG_TC_DANGKY",
        "HIST_PASS_RATE",
        "HIST_GPA_STD",
        "TOTAL_FAIL_CREDITS",
        "LAG1_GPA",
        "LAG2_GPA",
        "LAG1_PASS_RATE",
        "TREND_GPA",
        "LOAD_RATIO",
        "SCORE_GAP",
        "DIEM_TRUNGTUYEN",
        "DIEM_CHUAN",
        "NAM_TUYENSINH",
        "PTXT_1",
        "PTXT_100",
        "PTXT_200",
        "PTXT_3",
        "PTXT_402",
        "PTXT_409",
        "PTXT_5",
        "PTXT_500",
        "AVG_SCORE",
    ]

    for col in pred_features:
        if col not in pred_df.columns:
            pred_df[col] = 0

    ready_df = pred_df[pred_features].rename(columns={"TC_DANGKY_x": "TC_DANGKY"})

    # Align to FEATURES
    ready_df = ready_df.reindex(columns=FEATURES, fill_value=0)

    # Numeric safety
    for col in FEATURES:
        ready_df[col] = pd.to_numeric(ready_df[col], errors="coerce").fillna(0)

    return ready_df, pred_df

def predict_credits(
    model: XGBRegressor,
    ready_df: pd.DataFrame,
    tc_dangky: np.ndarray,
) -> Tuple[np.ndarray, np.ndarray]:
    pass_rate = model.predict(ready_df)
    pass_rate = np.clip(pass_rate, 0.0, 1.0)
    tc = np.asarray(tc_dangky, dtype=float)
    tc = np.nan_to_num(tc, nan=0.0, posinf=0.0, neginf=0.0)
    pred_tc = tc * pass_rate
    return pass_rate, pred_tc

# Paths
DATA_DIR = Path("data")
BEST_PARAMS_PATH = Path("best_params_xgboost.json")

def rmse(y_true, y_pred) -> float:
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

## 2) Load dữ liệu (local)
Cần đủ 3 file:
- `data/academic_records.csv`
- `data/admission.csv`
- `data/test.csv`

In [ ]:
academic_path = DATA_DIR / 'academic_records.csv'
admission_path = DATA_DIR / 'admission.csv'
test_path = DATA_DIR / 'test.csv'

for p in [academic_path, admission_path, test_path]:
    if not p.exists():
        raise FileNotFoundError(f'Missing file: {p}')

academic_df = pd.read_csv(academic_path)
admission_df = pd.read_csv(admission_path)
test_df = pd.read_csv(test_path)

print('academic_df:', academic_df.shape)
print('admission_df:', admission_df.shape)
print('test_df:', test_df.shape)

display(academic_df.head())
display(admission_df.head())
display(test_df.head())

## 3) Feature engineering (đồng bộ `pipeline.py`)
Mục tiêu: tạo `main_df` gồm các feature lịch sử cho từng sinh viên theo từng học kỳ.
Trong project này, phần feature engineering **được chuẩn hoá trong** `pipeline.build_main_df()` để tránh lệch logic giữa notebook và app.

In [ ]:
main_df = build_main_df(academic_df, admission_df)
print('main_df:', main_df.shape)

# Sanity checks
missing = [c for c in FEATURES if c not in main_df.columns]
print('Missing feature cols:', missing)

display(main_df.head())

## 4) Chuẩn bị train/valid
Target đang train là `PASS_RATE` (tỷ lệ hoàn thành).
Khi suy ra số tín chỉ hoàn thành, ta dùng: `PRED_TC_HOANTHANH = TC_DANGKY * PASS_RATE`.

Để đánh giá hợp lý theo thời gian, ta giữ lại **1 học kỳ** làm validation (mặc định `HK2 2023-2024`). Nếu học kỳ này không tồn tại trong dữ liệu thì notebook sẽ tự lấy học kỳ mới nhất.

In [ ]:
def _key_hoc_ky(hoc_ky: str):
    # Format: 'HK1 2020-2021'
    hk, year = str(hoc_ky).split()
    hk = int(hk.replace('HK', ''))
    year = int(year.split('-')[0])
    return (year, hk)

preferred_valid_hk = os.environ.get('VALID_HK', 'HK2 2023-2024')
hoc_ky_values = main_df['HOC_KY'].astype(str).dropna().unique().tolist()

if preferred_valid_hk in hoc_ky_values:
    valid_hk = preferred_valid_hk
else:
    valid_hk = sorted(hoc_ky_values, key=_key_hoc_ky)[-1]

print('Using validation semester:', valid_hk)

X = main_df[FEATURES].copy()
y = main_df['PASS_RATE'].astype(float).copy()

valid_mask = main_df['HOC_KY'].astype(str) == str(valid_hk)
X_train, y_train = X.loc[~valid_mask], y.loc[~valid_mask]
X_valid, y_valid = X.loc[valid_mask], y.loc[valid_mask]

print('Train:', X_train.shape, 'Valid:', X_valid.shape)
print('PASS_RATE train mean:', float(y_train.mean()), 'valid mean:', float(y_valid.mean()))

## 5) Train baseline XGBoost (không tune)
Cell này giúp kiểm tra nhanh pipeline có chạy ổn không trước khi tune Optuna.

In [ ]:
baseline_params = {
    'n_estimators': 800,
    'learning_rate': 0.05,
    'max_depth': 6,
    'subsample': 0.85,
    'colsample_bytree': 0.85,
    'min_child_weight': 2,
    'reg_lambda': 1.0,
    'reg_alpha': 0.0,
    'gamma': 0.0,
    'tree_method': 'hist',  # CPU friendly
}

baseline_model = XGBRegressor(
    objective='reg:squarederror',
    n_jobs=-1,
    random_state=42,
    **baseline_params,
)

baseline_model.fit(
    X_train,
    y_train,
    eval_set=[(X_valid, y_valid)],
    verbose=False,
)

pred_valid = baseline_model.predict(X_valid)
print('Baseline RMSE:', rmse(y_valid, pred_valid))

## 6) (Tuỳ chọn) Tune XGBoost bằng Optuna
- Mặc định chạy nhanh với `N_TRIALS=30`. Có thể tăng lên 100-300 để tối ưu kỹ hơn.
- Nếu có GPU và XGBoost support CUDA, set biến môi trường `USE_CUDA=1` trước khi chạy kernel.
- Kết quả sẽ được lưu ra `best_params_xgboost.json` để Streamlit app dùng lại.

In [ ]:
N_TRIALS = int(os.environ.get('N_TRIALS', '30'))
USE_CUDA = os.environ.get('USE_CUDA', '0') == '1'

if optuna is None:
    raise ImportError('Optuna chưa được cài. Hãy chạy: pip install optuna (hoặc pip install -r requirements.txt)')

def suggest_xgb_params(trial: optuna.trial.Trial) -> dict:
    params = {
        'tree_method': 'hist',
        'n_estimators': trial.suggest_int('n_estimators', 300, 4000),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 0.95),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.95),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 1e-6, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.5, 10.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 5.0),
    }
    if USE_CUDA:
        # XGBoost >= 2.0
        params['device'] = 'cuda'
    return params

def objective(trial) -> float:
    params = suggest_xgb_params(trial)
    model = XGBRegressor(
        objective='reg:squarederror',
        n_jobs=-1,
        random_state=42,
        **params,
    )

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=False,
    )

    pred = model.predict(X_valid)
    return rmse(y_valid, pred)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=N_TRIALS)

print('[Optuna] Best RMSE:', study.best_value)
print('[Optuna] Best params:', study.best_params)

# Lưu best params để Streamlit/app dùng lại
with open(BEST_PARAMS_PATH, 'w', encoding='utf-8') as f:
    json.dump(study.best_params, f, ensure_ascii=False, indent=2)

print('Saved:', BEST_PARAMS_PATH)

## 7) Train final model trên toàn bộ dữ liệu
Cell này đọc `best_params_xgboost.json` (nếu có) rồi fit trên toàn bộ `main_df`.
Streamlit app cũng làm tương tự, nên bước này chủ yếu để bạn kiểm tra nhanh output ở notebook.

In [ ]:
best_params = {}
if BEST_PARAMS_PATH.exists():
    best_params = json.loads(BEST_PARAMS_PATH.read_text(encoding='utf-8'))

final_model = XGBRegressor(
    objective='reg:squarederror',
    n_jobs=-1,
    random_state=42,
    tree_method='hist',
    **best_params,
)

final_model.fit(X, y)
print('Final model fitted. best_params keys:', list(best_params.keys()))

## 8) Dự báo cho `data/test.csv` và xuất `submissionn.csv`
Logic tạo feature cho prediction được chuẩn hoá trong `pipeline.build_prediction_frame()` để đồng bộ với Streamlit app.

In [ ]:
ready_df, pred_df = build_prediction_frame(test_df, main_df, preferred_last_hk=os.environ.get('LAST_HK', 'HK2 2023-2024'))

tc_dangky = pred_df.get('TC_DANGKY_x', pred_df.get('TC_DANGKY', 0)).values
pass_rate, pred_tc = predict_credits(final_model, ready_df, tc_dangky=tc_dangky)

out = pd.DataFrame({
    'MA_SO_SV': pred_df['MA_SO_SV'],
    'PRED_PASS_RATE': pass_rate,
    'PRED_TC_HOANTHANH': pred_tc,
})

out_path = 'submissionn.csv'
out.to_csv(out_path, index=False)
print('Saved:', out_path, 'shape:', out.shape)
display(out.head())

## 9) Bước tiếp theo
Sau khi đã có `best_params_xgboost.json`, bạn chạy app:
```bash
streamlit run streamlit_app.py
```
App sẽ tự load lại dữ liệu + train model theo params và sinh SHAP note cho từng dòng dự đoán.